## Dividend Before Normalization

In [1]:
import os
import pandas as pd
import numpy as np 
import glob
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import MultipleLocator

## **Split 미반영 DataFrame 출력**

In [2]:
# 데이터 불러와
dividend_path = 'Data_Dividend'

all_files = os.listdir(dividend_path)
dividend_files = [file for file in all_files if file.endswith('_d.csv')]

# 최종 병합될 데이터프레임 초기화
merged_df = pd.DataFrame()

for file in dividend_files:
    ticker = file.split('_')[0]  # 파일명에서 티커 이름만 추출
    file_path = os.path.join(dividend_path, file)
    df = pd.read_csv(file_path)

    # 'Dividends' 열 이름을 티커 이름으로 변경
    df.rename(columns={'Dividends': ticker}, inplace=True)

    if merged_df.empty:
        merged_df = df  # 첫 번째 데이터프레임을 기준으로 설정
    else:
        # 날짜 기준 병합(결측값 갖는 날짜여도 상관 ㄴㄴ하도록)
        merged_df = pd.merge(merged_df, df, on='Date', how='outer')

# 날짜 기준 정렬
merged_df.sort_values(by='Date', inplace=True)

# 날짜 열 제외하고 티커명에 해당하는 열만 알파벳 순 정렬
sorted_columns = sorted(merged_df.columns[1:])  # 날짜 열(Date) 제외하고 정렬

# 날짜 열을 1열로 + 정렬된 열 순서를 적용
sorted_df = merged_df[['Date'] + sorted_columns]

# 출력
sorted_df


,Date,ABBV,ABT,ADM,ADP,AFL,ALB,AMCR,AOS,APD,...,SHW,SJM,SPGI,SWK,SYY,TGT,TROW,WMT,WST,XOM
5711,1962-01-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5525,1962-02-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01875
4235,1962-02-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4047,1962-03-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5712,1962-04-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215,2024-03-14,NaN,NaN,NaN,NaN,NaN,0.4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.24,0.208,NaN,NaN
648,2024-03-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024,2024-03-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2198,2024-03-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## **월별로 묶어보자**

In [3]:
# 'Date' 열을 datetime 타입으로 변환
sorted_df['Date'] = pd.to_datetime(sorted_df['Date'])

# 연도와 월만 추출하여 새로운 'YearMonth' 열 생성
sorted_df['YearMonth'] = sorted_df['Date'].dt.to_period('M')

# 'YearMonth' 열을 기준으로 그룹화하고, 'Date' 열을 제외하고 나머지 회사들의 합계를 계산
# 'numeric_only=True'를 설정하여 숫자형 열에만 합계 연산을 적용
monthly_dividends = sorted_df.drop(columns=['Date']) \
                             .groupby('YearMonth', as_index=False) \
                             .sum(numeric_only=True)

# 결과 확인
monthly_dividends

,YearMonth,ABBV,ABT,ADM,ADP,AFL,ALB,AMCR,AOS,APD,...,SHW,SJM,SPGI,SWK,SYY,TGT,TROW,WMT,WST,XOM
0,1962-01,0.00,0.00,0.00,0.0,0.00,0.0,0.000,0.00,0.00,...,0.000,0.00,0.00,0.00,0.0,0.0,0.00,0.000,0.0,0.00000
1,1962-02,0.00,0.00,0.00,0.0,0.00,0.0,0.000,0.00,0.00,...,0.000,0.00,0.00,0.00,0.0,0.0,0.00,0.000,0.0,0.01875
2,1962-03,0.00,0.00,0.00,0.0,0.00,0.0,0.000,0.00,0.00,...,0.000,0.00,0.00,0.00,0.0,0.0,0.00,0.000,0.0,0.00000
3,1962-04,0.00,0.00,0.00,0.0,0.00,0.0,0.000,0.00,0.00,...,0.000,0.00,0.00,0.00,0.0,0.0,0.00,0.000,0.0,0.00000
4,1962-05,0.00,0.00,0.00,0.0,0.00,0.0,0.000,0.00,0.00,...,0.000,0.00,0.00,0.00,0.0,0.0,0.00,0.000,0.0,0.01875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,2023-11,0.00,0.00,0.45,0.0,0.42,0.0,0.125,0.00,0.00,...,0.605,1.06,0.90,0.81,0.0,1.1,0.00,0.000,0.2,0.95000
732,2023-12,0.00,0.00,0.00,1.4,0.00,0.4,0.000,0.00,1.75,...,0.000,0.00,0.00,0.00,0.0,0.0,1.22,0.190,0.0,0.00000
733,2024-01,1.55,0.55,0.00,0.0,0.00,0.0,0.000,0.32,0.00,...,0.000,0.00,0.00,0.00,0.5,0.0,0.00,0.000,0.2,0.00000
734,2024-02,0.00,0.00,0.50,0.0,0.50,0.0,0.125,0.00,0.00,...,0.715,1.06,0.91,0.00,0.0,1.1,0.00,0.000,0.0,0.95000


In [4]:
monthly_dividends.columns

Index(['YearMonth', 'ABBV', 'ABT', 'ADM', 'ADP', 'AFL', 'ALB', 'AMCR', 'AOS',
       'APD', 'ATO', 'BDX', 'BEN', 'BF-B', 'BRO', 'CAH', 'CAT', 'CB', 'CHD',
       'CHRW', 'CINF', 'CL', 'CLX', 'CTAS', 'CVX', 'DOV', 'ECL', 'ED', 'EMR',
       'ESS', 'EXPD', 'FRT', 'GD', 'GPC', 'GWW', 'HRL', 'IBM', 'ITW', 'JNJ',
       'KMB', 'KO', 'LEG', 'LIN', 'LOW', 'MCD', 'MDT', 'MKC', 'MMM', 'NDSN',
       'NEE', 'NUE', 'O', 'PEP', 'PG', 'PNR', 'PPG', 'ROP', 'SHW', 'SJM',
       'SPGI', 'SWK', 'SYY', 'TGT', 'TROW', 'WMT', 'WST', 'XOM'],
      dtype='object')

In [5]:
df = monthly_dividends.copy()

In [14]:
df.rename(columns={'YearMonth': 'Date'}, inplace=True)


# 2014년 1월 1일 이후의 데이터만 필터링
df_filtered = df[(df['Date'] >= '2014-02') & (df['Date'] <= '2024-02')]

df_filtered

,Date,ABBV,ABT,ADM,ADP,AFL,ALB,AMCR,AOS,APD,...,SHW,SJM,SPGI,SWK,SYY,TGT,TROW,WMT,WST,XOM
614,2014-02,0.00,0.00,0.24,0.000000,0.185,0.000,0.176,0.075,0.000000,...,0.183333,0.58,0.30,0.00,0.00,0.43,0.00,0.00,0.0,0.63
615,2014-03,0.00,0.00,0.00,0.421422,0.000,0.275,0.000,0.000,0.712303,...,0.000000,0.00,0.00,0.50,0.00,0.00,0.44,0.16,0.0,0.00
616,2014-04,0.42,0.22,0.00,0.000000,0.000,0.000,0.000,0.075,0.000000,...,0.000000,0.00,0.00,0.00,0.29,0.00,0.00,0.00,0.1,0.00
617,2014-05,0.00,0.00,0.24,0.000000,0.185,0.000,0.000,0.000,0.000000,...,0.183333,0.58,0.30,0.00,0.00,0.43,0.00,0.16,0.0,0.69
618,2014-06,0.00,0.00,0.00,0.421422,0.000,0.275,0.000,0.000,0.712303,...,0.000000,0.00,0.00,0.50,0.00,0.00,0.44,0.00,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,2023-10,1.48,0.51,0.00,0.000000,0.000,0.000,0.000,0.320,0.000000,...,0.000000,0.00,0.00,0.00,0.50,0.00,0.00,0.00,0.0,0.00
731,2023-11,0.00,0.00,0.45,0.000000,0.420,0.000,0.125,0.000,0.000000,...,0.605000,1.06,0.90,0.81,0.00,1.10,0.00,0.00,0.2,0.95
732,2023-12,0.00,0.00,0.00,1.400000,0.000,0.400,0.000,0.000,1.750000,...,0.000000,0.00,0.00,0.00,0.00,0.00,1.22,0.19,0.0,0.00
733,2024-01,1.55,0.55,0.00,0.000000,0.000,0.000,0.000,0.320,0.000000,...,0.000000,0.00,0.00,0.00,0.50,0.00,0.00,0.00,0.2,0.00


In [15]:
output_path = 'Dividend Before Normalization.csv' 
df_filtered.to_csv(output_path, index=True) 